In [301]:
import re

class Gate():
    def __init__(self,inputs,result,operator):
        self.inputs = inputs
        self.result = result
        self.operator = operator

    def process(self,bool1,bool2):
        if self.operator == 0:
            return bool1 & bool2
        elif self.operator == 1:
            return bool1 | bool2
        elif self.operator == 2:
            return bool1 ^ bool2
        else:
            return -1

with open('input.txt') as file:
    data = file.read()

data = data.split('\n\n')

data_inputs = data[0].split('\n')

memory = {}
for row in data_inputs:
    pos, value = row.split(': ')
    memory[pos] = value == '1'

data_gates = data[1].split('\n')

gates = {}
operator_mapping = {'AND':0,'OR':1,'XOR':2}
for gate in data_gates:
    operator = re.search('AND|OR|XOR',gate).group()
    left, result = gate.split(' -> ')
    left, right = left.split(' ' + operator + ' ')
    gates[result] = Gate((left,right),result,operator_mapping[operator])

In [ ]:
def solve_gate(gate,gates,depth=0):
    inputs = gate.inputs
    for gate_input in inputs:
        if gate_input not in memory:
            solve_gate(gates[gate_input],gates,depth+1)
        

    output = gate.process(memory[inputs[0]], memory[inputs[1]])
    memory[gate.result] = output
    return output

final_output = sorted([key for key in gates if key[0] == 'z'], reverse=True)

s = ''
for output in final_output:
    s += str(solve_gate(gates[output], gates) * 1)

int(s,2)
s


In [ ]:
# This cell is pure visualisation

from itertools import combinations
ii = 0

def find_nodes(gate_name, gates):

    process_que = [(gate_name,0)]
    visited = []
    while len(process_que) > 0:
        sorted(process_que, key = lambda node: node[1])
        gate_name,depth = process_que.pop(0)
        if gate_name[0] in ('x','y'):
            visited.append((gate_name, depth, -1))
            continue
        else:
            
            visited.append((gate_name, depth, gates[gate_name].operator))

        gate = gates[gate_name]
        for gate_input in gate.inputs:
            process_que.append((gate_input,depth+1))

    return visited


for ii in range(len(final_output)-1):
    visited_prev = find_nodes(f'z{ii:02d}',gates_temp)
    visited = find_nodes(f'z{ii+1:02d}',gates_temp)

    for result,depth,operator in visited_prev:
        if result[0] != 'z' and (result, depth+2,operator) not in visited:
            print(ii,(result,depth+2,operator), visited)
            break


In [ ]:
from collections import defaultdict
next_gate = defaultdict(set)

for gate_name in gates:
    inp1,inp2 = gates[gate_name].inputs
    next_gate[inp1].add(gates[gate_name].operator)
    next_gate[inp2].add(gates[gate_name].operator)
next_gate

In [ ]:
# This was horrible
from collections import Counter
errors = []
categories = {}
next_gate_categories = {():0,(1,):1,(0,2):2}

for ii in range(3):
    categories[ii] = [[0 for ii in range(3)] for jj in range(6)]

gate_cateogories = Counter()
for gate_name in gates:
    gate = gates[gate_name]
    inp1,inp2 = gate.inputs
    out = gate.result
    gate_cat = (inp1[0] in ['x','y'], inp2[0] in ['x','y'], gate.operator, out[0] == 'z', tuple(next_gate[gate.result]))
    if gate.result not in ['z00','z45']:
        gate_cateogories[gate_cat] += 1

outlier_categories = [category for category in gate_cateogories if gate_cateogories[category] < 5]


outliers = []
for gate_name in gates:
    gate = gates[gate_name]
    inp1,inp2 = gate.inputs
    out = gate.result
    gate_cat = (inp1[0] in ['x','y'], inp2[0] in ['x','y'], gate.operator, out[0] == 'z', tuple(next_gate[gate.result]))
    if gate_cat in outlier_categories and gate.result not in ['z00','z45','gsv']:
        outliers.append(gate.result)
        print(gate.result, gate.inputs, gate_cat, gate_cateogories[gate_cat])


','.join(sorted(outliers))
